In [1]:
import regex as re
import pandas as pd

In [2]:
with open("uu8-1999.txt") as file:
    file = file.read()

## Remove Header

In [4]:
file_no_header = re.sub("\\nPRESIDEN\\nREPUBLIK INDONESIA\\n- \p{N}+ -", " ", file, flags=re.DOTALL)
file_no_header = re.sub("(\\n[^\\n]*? \…[^\\n]*)(?=\\n)", " ", file_no_header, flags=re.DOTALL)

## Pisah Penjelasan

In [6]:
penjelasan = re.search("PRESIDEN\\nREPUBLIK INDONESIA\\nPENJELASAN.*", file_no_header, re.DOTALL).group()
badan = file_no_header.replace(penjelasan, "")

## Ambil BAB

In [8]:
def ambil_bab(text):
    pattern = "(?<=\\n)(BAB [IVXLCDM]+.*?)(?=\\nBAB|$)"
    bab = re.findall(pattern, text, re.DOTALL)
    if bab:
        ## ambil nomor dan nama bab
        pattern_no = "(?<=BAB)(?:\s*)([IVXLCDM]+)"
        pattern_nama = "(?<=BAB)(?:\s*[IVXLCDM]+\s*\\n)([\p{Lu}\s\\n]*)(?=\\n)"
        pattern_isi = "(?<=BAB)(?:\s*[IVXLCDM]+\s*\\n[\p{Lu}\s\\n]*)(?:\\n)(.*)"
        no = [re.search(pattern_no, i, re.DOTALL).group(1) for i in bab]
        nama = [re.search(pattern_nama, i, re.DOTALL).group(1) for i in bab]
        isi = [re.search(pattern_isi, i, re.DOTALL).group(1) for i in bab]
        return no, nama, isi
    else:
        no = [""]
        nama = [""]
        isi = [text]
        return no, nama, isi

In [9]:
no_bab, nama_bab, isi_bab = ambil_bab(badan)
df_bab = {"no_bab": no_bab, "nama_bab": nama_bab, "isi_bab": isi_bab}

In [10]:
df_bab = pd.DataFrame(df_bab)
df_bab

,no_bab,nama_bab,isi_bab
0,I,KETENTUAN UMUM,Pasal 1\nDalam Undang-undang ini yang dimaksud...
1,II,ASAS DAN TUJUAN,Pasal 2\nPerlindungan konsumen berasaskan manf...
2,III,HAK DAN KEWAJIBAN,Bagian Pertama\nHak dan Kewajiban Konsumen\nPa...
3,IV,PERBUATAN YANG DILARANG\nBAGI PELAKU USAHA,Pasal 8\n(1) Pelaku usaha dilarang memproduksi...
4,V,KETENTUAN PENCANTUMAN KLAUSULA BAKU,Pasal 18\n(1) Pelaku usaha dalam menawarkan ba...
5,VI,TANGGUNG JAWAB PELAKU USAHA,Pasal 19\n(1) Pelaku usaha bertanggung jawab m...
6,VII,PEMBINAAN DAN PENGAWASAN,Bagian Pertama\nPembinaan\nPasal 29\n(1) Pemer...
7,VIII,BADAN PERLINDUNGAN KONSUMEN NASIONAL,"Bagian Pertama \nNama, Kedudukan, Fungsi, dan ..."
8,IX,LEMBAGA PERLINDUNGAN KONSUMEN\nSWADAYA MASYARAKAT,Pasal 44\n(1) Pemerintah mengakui lembaga perl...
9,X,PENYELESAIAN SENGKETA,Bagian Pertama\nUmum\nPasal 45\n(1) Setiap kon...


## Ambil Bagian

In [12]:
def ambil_bagian(text):
    pattern = "(?<=\\n|^)(Bagian \p{Lu}.*?)(?=\\nBagian \p{Lu}|$)"
    bagian = re.findall(pattern, text, re.DOTALL)
    if bagian:
        pattern_nama = "(?<=Bagian \p{Lu})(?:\p{L}*\s*\\n)(.*?)(?=\\nPasal)"
        pattern_isi = "Pasal.*"
        nama = [re.search(pattern_nama, i, re.DOTALL).group(1) for i in bagian]
        isi = [re.search(pattern_isi, i, re.DOTALL).group(0) for i in bagian]
        return nama, isi
    else:
        nama = [""]
        isi = [text]
        return nama, isi

In [13]:
for i in range(len(df_bab)):
    nama_bagian, isi_bagian = ambil_bagian(df_bab["isi_bab"][i])
    df_bagian = {"nama_bagian": nama_bagian, "isi_bagian": isi_bagian}
    display(pd.DataFrame(df_bagian))

,nama_bagian,isi_bagian
0,,Pasal 1\nDalam Undang-undang ini yang dimaksud...


,nama_bagian,isi_bagian
0,,Pasal 2\nPerlindungan konsumen berasaskan manf...


,nama_bagian,isi_bagian
0,Hak dan Kewajiban Konsumen,Pasal 4\nHak konsumen adalah:\na. hak atas ken...
1,Hak dan Kewajiban Pelaku Usaha,Pasal 6\nHak pelaku usaha adalah:\na. hak untu...


,nama_bagian,isi_bagian
0,,Pasal 8\n(1) Pelaku usaha dilarang memproduksi...


,nama_bagian,isi_bagian
0,,Pasal 18\n(1) Pelaku usaha dalam menawarkan ba...


...

,nama_bagian,isi_bagian
0,,Pasal 49\n(1) Pemerintah membentuk badan penye...


,nama_bagian,isi_bagian
0,,Pasal 59\n(1) Selain Pejabat Polisi Negara Rep...


,nama_bagian,isi_bagian
0,Sanksi Administratif,Pasal 60\n(1) Badan penyelesaian sengketa kons...
1,Sanksi Pidana,Pasal 61\nPenuntutan pidana dapat dilakukan te...


,nama_bagian,isi_bagian
0,,Pasal 64\nSegala ketentuan peraturan perundang...


,nama_bagian,isi_bagian
0,,Pasal 65\nUndang-undang ini mulai berlaku sete...


In [14]:
nama_bagian, isi_bagian = ambil_bagian(df_bab["isi_bab"][7])
df_bagian = {"nama_bagian": nama_bagian, "isi_bagian": isi_bagian}
pd.DataFrame(df_bagian)

,nama_bagian,isi_bagian
0,"Nama, Kedudukan, Fungsi, dan Tugas",Pasal 31\nDalam rangka mengembangkan upaya per...
1,Susunan Organisasi dan Keanggotaan,Pasal 35\n(1) Badan Perlindungan Konsumen Nasi...


## Ambil Pasal

In [16]:
def ambil_pasal(text):
    pattern = "(?<=\\n|^)(Pasal \p{N}+\s*\\n.*?)(?=\\nPasal|$)"
    pasal = re.findall(pattern, text, re.DOTALL)
    if pasal:
        pattern_no = "(?<=Pasal)(?:\s*)(\p{N}+)"
        pattern_isi = "(?<=Pasal)(?:\s*\p{N}+\s*\\n)(.*)"
        no = [re.search(pattern_no, i, re.DOTALL).group(1) for i in pasal]
        isi = [re.search(pattern_isi, i, re.DOTALL).group(1) for i in pasal]
        return no, isi
    else:
        no = [""]
        isi = [text]
        return no, isi

In [17]:
for i in range(len(df_bagian)):
    no_pasal, isi_pasal = ambil_pasal(df_bagian["isi_bagian"][i])
    df_pasal = {"no_pasal": no_pasal, "isi_pasal": isi_pasal}
    display(pd.DataFrame(df_pasal))

,no_pasal,isi_pasal
0,31,Dalam rangka mengembangkan upaya perlindungan ...
1,32,Badan Perlindungan Konsumen Nasional berkedudu...
2,33,Badan Perlindungan Konsumen Nasional mempunyai...
3,34,(1) Untuk menjalankan fungsi sebgaimana dimaks...


,no_pasal,isi_pasal
0,35,(1) Badan Perlindungan Konsumen Nasional terdi...
1,36,Anggota Badan Perlindungan Konsumen Nasional t...
2,37,Persyaratan keanggotaan Badan Perlindungan Kon...
3,38,Keanggotaan Badan Perlindungan Konsumen Nasion...
4,39,"(1) Untuk kelancaran pelaksanaan tugas, Badan ..."
5,40,(1) Apabila diperlukan Badan Perlindungan Kons...
6,41,"Dalam pelaksanaan tugas, Badan Perlindungan Ko..."
7,42,Biaya untuk pelaksanaan tugas Badan Perlindung...
8,43,Ketentuan lebih lanjut mengenai pembentukan Ba...


In [18]:
no_pasal, isi_pasal = ambil_pasal(df_bagian["isi_bagian"][0])
df_pasal = {"no_pasal": no_pasal, "isi_pasal": isi_pasal}

In [19]:
df_pasal = pd.DataFrame(df_pasal)
df_pasal

,no_pasal,isi_pasal
0,31,Dalam rangka mengembangkan upaya perlindungan ...
1,32,Badan Perlindungan Konsumen Nasional berkedudu...
2,33,Badan Perlindungan Konsumen Nasional mempunyai...
3,34,(1) Untuk menjalankan fungsi sebgaimana dimaks...


## Ambil Ayat

In [21]:
def ambil_ayat(text):
    clean_pasal = re.sub('(?<=ayat )(\\n)(?=\(\p{N}\))', "", text, flags=re.DOTALL)
    ayat = re.findall('(?<=\\n|^)(\(\p{N}+\).*?)(?=\\n\(\p{N}\)|$)', text, flags=re.DOTALL)
    
    if ayat:
        pattern_no = "(?<=\\n|^)(?:\()(\p{N}+)(?=\))"
        pattern_isi = "(?<=\(\p{N}+\)\s*)(.*)"
        no = [re.search(pattern_no, i, re.DOTALL).group(1) for i in ayat]
        isi = [re.search(pattern_isi, i, re.DOTALL).group(0) for i in ayat]
        
        return no, isi
    else:
        no = [""]
        isi = [text]
        return no, isi

In [22]:
for i in range(len(df_pasal)):
    no_ayat, isi_ayat = ambil_ayat(df_pasal["isi_pasal"][i])
    df_ayat = {"no_ayat": no_ayat, "isi_ayat": isi_ayat}
    display(pd.DataFrame(df_ayat))

,no_ayat,isi_ayat
0,,Dalam rangka mengembangkan upaya perlindungan ...


,no_ayat,isi_ayat
0,,Badan Perlindungan Konsumen Nasional berkedudu...


,no_ayat,isi_ayat
0,,Badan Perlindungan Konsumen Nasional mempunyai...


,no_ayat,isi_ayat
0,1,Untuk menjalankan fungsi sebgaimana dimaksud ...
1,2,Dalam melaksanakan tugas sebagaimana dimaksud...


In [23]:
no_ayat, isi_ayat = ambil_ayat(df_pasal["isi_pasal"][3])
df_ayat = {"no_ayat": no_ayat, "isi_ayat": isi_ayat}

In [24]:
pd.DataFrame(df_ayat)

,no_ayat,isi_ayat
0,1,Untuk menjalankan fungsi sebgaimana dimaksud ...
1,2,Dalam melaksanakan tugas sebagaimana dimaksud...
